In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import os
import pandas as pd
import glob
from radprompter import RadPrompter, Prompt, vLLMClient, OllamaClient

In [7]:
report_files = glob.glob("sample_reports/*.txt")

In [8]:
reports = []
report_ids = []
for report_file in report_files:
    report_ids.append(os.path.basename(report_file))
    with open(report_file, 'r') as f:
        reports.append(f.read())

In [9]:
prompt = Prompt('demo_prompt_no_CoT.toml')
# prompt = Prompt('sample_prompt.toml')

In [10]:
prompt

In [12]:
prompt.schemas[2]

In [20]:
client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:8000/v1",
    temperature = 0.0,
    seed=42
)

In [21]:
engine = RadPrompter(
    client=client,
    prompt=prompt,
    concurrency=12,
    hide_blocks=True,
    output_file="output.csv",
)

In [39]:
sample = [{'report': sample_report} for sample_report in reports]

In [40]:
sample = [{'report': sample_report, 'report_id': report_id} for sample_report, report_id in zip(reports, report_ids)]

In [44]:
sample

[{'report': 'Clinical Information:\n72-year-old female with sudden onset shortness of breath and hypoxia. Evaluate for pulmonary embolism.\nTechnique:\nCT pulmonary angiography with IV contrast was performed.\nFindings:\nThere is a large saddle embolus extending from the main pulmonary artery into the right and left main pulmonary arteries. Multiple filling defects are also seen within the segmental and subsegmental branches bilaterally, indicating additional pulmonary emboli.\nThe right ventricle appears dilated at 4.6 cm, greater than the left ventricle. There is flattening of the interventricular septum, suggesting right heart strain.\nMosaic perfusion is present within the lungs bilaterally, likely representing pulmonary infarcts.\nSmall bilateral pleural effusions are noted.\nImpression:\n\nSaddle pulmonary embolism involving the main pulmonary artery, right and left main branches, and multiple segmental/subsegmental branches.\nFindings of right heart strain with right ventricular

In [49]:
engine(sample)

Processing items: 100%|██████████| 3/3 [00:01<00:00,  1.94it/s]


In [51]:
results = pd.read_csv("output.csv", index_col=0)
results.head()

,Pulmonary Embolism_response,Left_response,Right_response,Acute_response,Chronic_response,RightHeartStrain_response,PulmonaryArteryHypertension_response,report,report_id
index,,,,,,,,,
0,Present,Yes,Yes,Yes,No,Yes,No,Clinical Information:\n72-year-old female with...,sample_report_2.txt
1,Present,Yes,Yes,No,No,No,No,Here is an example radiology report describing...,sample_report_3.txt
2,Present,Yes,Yes,No,Yes,No,Yes,Clinical Information:\n67-year-old male with s...,sample_report_1.txt


In [52]:
engine.save_log("demo_no_CoT.log")